In [1]:
import pandas as pd
from dateutil import parser
import pdfminer
from pdfminer.high_level import extract_text
import glob
import nltk
from string import punctuation
from nltk.corpus import stopwords
import re
from nltk.tag import pos_tag
from names_dataset import NameDataset
nd = NameDataset()
import spacy #numpy needs to be <1.20
nlp = spacy.load("en_core_web_sm")
import os
import pdfplumber
import pandas as pd
import datefinder
from datetime import datetime
from thefuzz import fuzz
# from pyresparser import ResumeParser
import numpy as np

C:\Users\rtd91\anaconda3\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
parse_resume_df = pd.read_csv(r'C:/Users/rtd91/Data/resume_samples_output_data/matched_resumes.csv')
gt_df = pd.read_csv(r'C:/Users/rtd91/Data/resume_samples_output_data/Ground Truth-v3.csv', index_col=False)

In [5]:
parse_resume_df

,index,name,name_ner,name_pos,name_ref,phone_number,email,master_degree,bachelor_degree,phd_degree,master_degree2,bachelor_degree2,phd_degree2,master_school,bachelor_school,phd_school,master_graduation_date,bachelor_graduation_date,phd_graduation_date
0,721091408_phonescreening.pdf,['Karthik Ramanarayana'],[],"['MD Contact', 'No Email', 'Avenue Baltimore',...",[],(410)-292-1151,['karthikr2194@gmail.com'],{'Master of Professional Studies in Data Scien...,{'Bachelor of Engineering in Electronics and C...,set(),Master of Professional Studies in Data Science...,Bachelor of Engineering in Electronics and Com...,NaN,"University of Maryland, Baltimore County, Balt...","Institute of Technology, Bangalore, India",NaN,['06/2022'],NaN,[]
1,821101104_phonescreening.pdf,['HARSH PUNDIR'],['HARSH PUNDIR'],"['Dr Hyattsville', 'Tulane Dr', 'PUNDIR Tulane...",['HARSH PUNDIR'],NaN,['hpundir@umd.edu'],{'Master of Science in Business Analytics'},{'Bachelor of Technology in Mechanical Enginee...,set(),"Master of Science in Business Analytics, Focus...",Bachelor of Technology in Mechanical Engineeri...,NaN,"School of Business, University of Maryland, Co...",School of Bus,NaN,['06/2022'],['06/2022'],[]
2,821101136_phonescreening.pdf,['TIRTH PATEL'],[],"['Holbrook NY', 'TIRTH PATEL', 'PATEL Holbrook']",['TIRTH PATEL'],469-370-9437,['tirth2410@gmail.com'],{'Master of Science'},{'Bachelor of Technology'},set(),"Master of Science, Business Analytics","Bachelor of Technology, Instrumentation and Co...",NaN,"University of Texas at Dallas, Dallas, TX","University of Texas at Dallas, Dallas, TX",NaN,['07/2016'],['07/2016'],[]
3,821110120.pdf,['Akanksha Bapna'],['Akanksha Bapna'],"['Akanksha Bapna', 'Authorization Eligible', '...",[],NaN,[],"{'Master of Marketing Analytics', 'Masters Mar...",{'Bachelors in Technology'},set(),Master of keting Analytics,"Bachelors in Technology, Chemical Engineering ...",NaN,"University of Maryland, Robert H. Smith School...",School of Bus,NaN,['03/2022'],['05/2018'],[]
4,821110440_Mythri.pdf,['Mythri Partha'],[],"['TX Phone', 'Partha Houston', 'Houston TX', '...",['Partha Houston'],(281) 725-7080,['mythripartha8@gmail.com'],set(),{'B.S. Chemistry - Minor: Mathematics'},set(),NaN,B S Chemistry - Minor: Mathematics,NaN,"University of Houston, Houston, TX","University of Houston, Houston, TX",NaN,NaN,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,YunhanDong.pdf,['Yunhan Zoe'],[],"['Zoe Dong', 'NY EDUCATION', 'Yunhan Zoe', 'Yo...",['wwwzoedongwixsitecomzoedongwwwlinkedincominy...,(319) 512-0421,['yd2578@columbia.edu'],{'M.S. in Computer and Information Science - A...,{'B.A. in Business Analytics and Information S...,set(),M S in Computer and Information Science - App...,NaN,NaN,University,University of Iowa,NaN,[],NaN,[]
197,ZhengyangZhang.pdf,['Jersey City'],[],"['Jersey City', 'NJ zzyygithubio', 'City NJ', ...",['Zhengyang Zhang'],(646) 531-8942,['zzhan67@stevens.edu'],{'Master (Master of Science in Data Science) G...,{'Bachelor (Bachelor of Arts in Statistics)'},set(),Master (Master of Science in Data Science) :,Bachelor (Bachelor of Arts in Statistics),NaN,"School of Data Science, Data Analyst ...",NaN,NaN,['06/2022'],['05/2020'],[]
198,ZhuohanZhang.pdf,['Zhuohan Zhang'],['Zhuohan Zhang \n '],"['NY New', 'New York', 'York NY', 'Zhuohan Zha...","['Zhuohan Zhang', 'New York']",(617) 763-4795,['zz2751@columbia.edu'],{'Master of Arts'},{'Bachelor of Arts'},set(),"Master of Arts, Statistics Expected","Bachelor of Arts, Mathematics (Statistics), Ec...",NaN,University,University,NaN,['12/2021'],['06/2022'],[]
199,ZixiaoHuang.pdf,['Zixiao Huang'],['Zixiao Huang \n\nzixiao.huang.74@gmail.com'],"['EDUCATION Northwestern', 'Zixiao Huang', 'Ev...",['Zixiao Huang'],615-977-2160,['zixiao.huang.74@gmail.com'],{'Master of Science in Analytics; GPA: 3.98/4....,{'Bachelor’s Degree in Mathematics and Compute...,set(),Master of Science in Analytics; : ...,Bachelor’s Degree in Mathematics and Computer ...

In [8]:
compared_df = pd.merge(gt_df, parse_resume_df, how = 'left', on = 'index')

In [9]:
compared_df.shape

(182, 31)

In [10]:
compared_df.columns

Index(['index', 'full_name', 'phd_school_x', 'phd_degree_x',
       'phd_graduation_date_x', 'master_school_x', 'master_degree_x',
       'master_graduation_date_x', 'bachelor_school_x', 'bachelor_degree_x',
       'bachelor_graduation_date_x', 'email_x', 'phone', 'name', 'name_ner',
       'name_pos', 'name_ref', 'phone_number', 'email_y', 'master_degree_y',
       'bachelor_degree_y', 'phd_degree_y', 'master_degree2',
       'bachelor_degree2', 'phd_degree2', 'master_school_y',
       'bachelor_school_y', 'phd_school_y', 'master_graduation_date_y',
       'bachelor_graduation_date_y', 'phd_graduation_date_y'],
      dtype='object')

In [11]:
def unlist(x):
    if isinstance(x, list) and len(x) != 0:
        for n in x:
            return n
    if x is None:
        return ''
    else:
        return ''

# -------------------- NAME -----------------------------------------------------------

compared_df['full_name_true'] = compared_df['full_name'].fillna('')
compared_df['full_name_true'].replace(to_replace=[None], value='', inplace=True)
compared_df['full_name_true'] = compared_df.full_name_true.apply(lambda x: str(x).capitalize())


compared_df['name_ref'] = compared_df.name.apply(lambda x:unlist(x))
compared_df['name_ref'] = compared_df['name'].fillna('')
compared_df['name_ref'].replace(to_replace=[None], value='', inplace=True)
compared_df['name_ref'] = compared_df.name.apply(lambda x:str(x).capitalize())

"""
compared_df.columns
compared_df['unlisted_name_ner'] = compared_df.name_ner.apply(lambda x:unlist(x))
compared_df['unlisted_name_ner'] = compared_df['unlisted_name_ner'].fillna('')
compared_df['unlisted_name_ner'].replace(to_replace=[None], value='', inplace=True)
compared_df['unlisted_name_ner'] = compared_df.unlisted_name_ner.apply(lambda x:str(x).capitalize())

compared_df['unlisted_name_pos'] = compared_df.name_pos.apply(lambda x:unlist(x))
compared_df['unlisted_name_pos'] = compared_df['unlisted_name_pos'].fillna('')
compared_df['unlisted_name_pos'].replace(to_replace=[None], value='', inplace=True)
compared_df['unlisted_name_pos'] = compared_df.unlisted_name_pos.apply(lambda x:str(x).capitalize())

compared_df['unlisted_name_ref'] = compared_df.name_ref.apply(lambda x:unlist(x))
compared_df['unlisted_name_ref'] = compared_df['unlisted_name_ref'].fillna('')
compared_df['unlisted_name_ref'].replace(to_replace=[None], value='', inplace=True)
compared_df['unlisted_name_ref'] = compared_df.unlisted_name_ref.apply(lambda x:str(x).capitalize())"""

"\ncompared_df.columns\ncompared_df['unlisted_name_ner'] = compared_df.name_ner.apply(lambda x:unlist(x))\ncompared_df['unlisted_name_ner'] = compared_df['unlisted_name_ner'].fillna('')\ncompared_df['unlisted_name_ner'].replace(to_replace=[None], value='', inplace=True)\ncompared_df['unlisted_name_ner'] = compared_df.unlisted_name_ner.apply(lambda x:str(x).capitalize())\n\ncompared_df['unlisted_name_pos'] = compared_df.name_pos.apply(lambda x:unlist(x))\ncompared_df['unlisted_name_pos'] = compared_df['unlisted_name_pos'].fillna('')\ncompared_df['unlisted_name_pos'].replace(to_replace=[None], value='', inplace=True)\ncompared_df['unlisted_name_pos'] = compared_df.unlisted_name_pos.apply(lambda x:str(x).capitalize())\n\ncompared_df['unlisted_name_ref'] = compared_df.name_ref.apply(lambda x:unlist(x))\ncompared_df['unlisted_name_ref'] = compared_df['unlisted_name_ref'].fillna('')\ncompared_df['unlisted_name_ref'].replace(to_replace=[None], value='', inplace=True)\ncompared_df['unlisted_na

In [12]:
comp_name1 = compared_df.apply(lambda x: fuzz.ratio(x['full_name_true'], x['name_ref']), axis = 1)
np.mean(comp_name1) #73.14

60.142857142857146

In [15]:
# -------------------- PHONE NUMBER -----------------------------------------------------

# phone_number, phone
from string import punctuation

def clean_phone_number(x):
    if isinstance(x, str):
        return_x = ''.join([p for p in x if p not in punctuation]).replace(" ", "")
        #print(return_x)
        if len(return_x) != 10 and return_x.startswith('1'):
            return return_x[1:]
        else:
            return return_x
    if not isinstance(x, str):
        return ''

compared_df['number_pred'] = compared_df.phone_number.apply(lambda x:clean_phone_number(x))
compared_df['phone_ref'] = compared_df.phone.apply(lambda x:clean_phone_number(x))


comp_phone = compared_df.apply(lambda x: fuzz.ratio(x['number_pred'], x['phone_ref']), axis = 1)
np.mean(comp_phone) #83.46

81.43406593406593

In [16]:
# --------------- EMAIL ----------------------

def unlist(x):
    if isinstance(x, list) and len(x) != 0:
        for n in x:
            return n
    if x is None:
        return ''
    else:
        return ''

compared_df['email_pref'] = compared_df.email_x.apply(lambda x:str(x).lower().replace(" ", ""))
compared_df['email_true'] = compared_df.email_y.apply(lambda x: str(x).replace("['", '').replace("']", '').replace("[", '').replace("]", ''))
comp_email = compared_df.apply(lambda x: fuzz.ratio(x['email_pref'], x['email_true']), axis = 1)
np.mean(comp_email) #97.82

89.91758241758242

In [ ]:
# ---------------------------- EDUCATION (to be cleaned) ---------------------------------------------------------------------------------------
compared_df = pd.merge(df, gt_df, how = 'left', on = 'index')
compared_df.columns

compared_df.master_degree_x
compared_df.master_degree2

compared_df.master_degree_y
compared_df.Master_Degree_2
compared_df.Master_Degree_3

def clean_degree_set(set_x):
    if not isinstance(set_x, set):
        return ''
    else:
        x = unlist(list(set_x))
        print(x)
        if isinstance(x, str):
            return_x = ''.join([p for p in x if p not in punctuation])
            print(return_x)
            return re.sub(' +', ' ', return_x).lower()

def clean_degree_string(string_x):

    if isinstance(string_x, str):
        return_x = ''.join([p for p in string_x if p not in punctuation])
        print(return_x)
        return re.sub(' +', ' ', return_x).lower()
    else:
        return ''


compared_df['cleaned_master_degree_pred2'][196]
compared_df['cleaned_master_degree_pred1'][196]
compared_df['cleaned_master_degree_true1'][196]
compared_df.master_degree_x[196]
compared_df.master_degree2[196]
compared_df['cleaned_master_degree_true1'][196]

compared_df['cleaned_master_degree_pred1'] = compared_df.master_degree_x.apply(lambda x:clean_degree_set(x))
compared_df['cleaned_master_degree_pred2'] = compared_df.master_degree2.apply(lambda x:clean_degree_string(x))

compared_df['cleaned_master_degree_true1'] = compared_df.master_degree_y.apply(lambda x:clean_degree_string(x))
compared_df['cleaned_master_degree_true2'] = compared_df.Master_Degree_2.apply(lambda x:clean_degree_string(x))
compared_df['cleaned_master_degree_true3'] = compared_df.Master_Degree_3.apply(lambda x:clean_degree_string(x))

compared_df.Master_Degree_2[compared_df.Master_Degree_2.notnull()] #15
compared_df['cleaned_master_degree_true1'][15]
compared_df['cleaned_master_degree_true2'][15]

compared_df['cleaned_master_degree_pred2'][15]
compared_df['cleaned_master_degree_pred1'][15]

fuzz.ratio(compared_df['cleaned_master_degree_pred2'][15], compared_df['cleaned_master_degree_true1'][15])
fuzz.ratio(compared_df['cleaned_master_degree_true2'][15], compared_df['cleaned_master_degree_pred2'][15])

fuzz.token_sort_ratio(compared_df['cleaned_master_degree_pred2'][15], compared_df['cleaned_master_degree_true1'][15])
fuzz.token_sort_ratio(compared_df['cleaned_master_degree_true2'][15], compared_df['cleaned_master_degree_pred2'][15])

fuzz.token_set_ratio(compared_df['cleaned_master_degree_true1'][15], compared_df['cleaned_master_degree_pred2'][15])
fuzz.token_set_ratio(compared_df['cleaned_master_degree_true2'][15], compared_df['cleaned_master_degree_pred2'][15])

# ---------------------- compare to other packages -------------------------------------------------------------------
from resume_parser import resumeparse
# data = resumeparse.read_file('/path/to/resume/file')
# data = ResumeParser(file_path).get_extracted_data()

# temp_person = []
# tagged = nlp(sentence_txt[0:200]) #Comment: assume names exist in the top section
#     for word in tagged.ents:
#         if word.label_ == "PERSON":
#             temp_person.append(word.text)

In [18]:
compared_df[["phd_graduation_date_x","phd_graduation_date_y","master_graduation_date_x","master_graduation_date_y","bachelor_graduation_date_x","bachelor_graduation_date_y"]]

,phd_graduation_date_x,phd_graduation_date_y,master_graduation_date_x,master_graduation_date_y,bachelor_graduation_date_x,bachelor_graduation_date_y
0,NaN,[],Dec 2021,['06/2022'],May 2016,NaN
1,NaN,[],NaN,['06/2022'],June 2019,['06/2022']
2,NaN,[],May 2021,['07/2016'],May 2018,['07/2016']
3,NaN,[],Dec 2021,['03/2022'],May 2018,['05/2018']
4,NaN,[],NaN,NaN,May 2020,[]
...,...,...,...,...,...,...
177,NaN,[],Dec 2021,[],May 2020,NaN
178,NaN,[],December 2021,['06/2022'],May 2020,['05/2020']
179,NaN,[],December 2021,['12/2021'],May 2020,['06/2022']
180,NaN,[],Dec 2021,['12/2021'],May 2020,['05/2020']


In [19]:
"""compared_df.groupby("phd_graduation_date_x").size()"""

phd_graduation_date_x
2022        1
May 2021    1
dtype: int64

In [20]:
"""compared_df.groupby("phd_graduation_date_y").size()"""

phd_graduation_date_y
[]    178
dtype: int64

In [21]:
"""compared_df.groupby("master_graduation_date_x").size()"""

master_graduation_date_x
08/2021                          1
12/1/2021, December 2019         1
12/2021                          4
2018, 2012                       1
2020, 2018                       1
2021                             5
2021-01                          1
4/1/2021, May 2013               1
Apr 2022                         1
April 2017                       1
April 2021                       1
Aug 2021                         5
August 2021                      9
Dec 2021                        38
Dec 2021; June 2015              1
Dec 21                           1
Dec. 2021                        4
Dec.2021, May.2018, Jun.2016     1
Dec2021                          1
December 2021                   34
December 2021, June 2014         1
December 2021.                   1
December 2021; October 2016      1
Dec’21                           1
Feb2022                          1
Jan’22                           1
Jun 2021                         2
June 2021                     

In [22]:
"""compared_df.groupby("master_graduation_date_y").size()"""

master_graduation_date_y
['01/2019', '05/2021']                1
['01/2020', '12/2021']                2
['01/2021']                           1
['02/2022']                           1
['03/2022', '09/2015', '05/2019']     1
['03/2022']                           4
['04/2022']                           1
['05/2017']                           1
['05/2019']                           1
['05/2021']                           3
['05/2022']                           2
['06/0322']                           1
['06/2010', '03/2014']                1
['06/2015']                           1
['06/2017']                           1
['06/2022', '06/2022']                2
['06/2022', '12/2021']                1
['06/2022']                          22
['07/2016']                           1
['08/2018', '05/2021']                1
['08/2019', '05/2021']                3
['08/2019', '12/2021']                1
['08/2020', '12/2021']                1
['08/2021']                           3
['09/2019', '06

In [120]:
"""compared_df.groupby("bachelor_graduation_date_x").size()"""

bachelor_graduation_date_x
                  1
05/2018           1
05/2020           2
06/2017           2
12/2021           1
                 ..
May’17            1
May’21            1
Nov 2017          1
September 2019    1
‘21               1
Length: 71, dtype: int64

In [119]:
"""compared_df.groupby("bachelor_graduation_date_y").size()"""

bachelor_graduation_date_y
['01/2020', '12/2021']                           1
['02/2019']                                      1
['03/2017']                                      1
['03/2022']                                      3
['04/2022']                                      1
['05/2013', '06/2017']                           1
['05/2013']                                      1
['05/2014']                                      1
['05/2017']                                      2
['05/2018']                                      2
['05/2019']                                      3
['05/2020']                                      5
['05/2021']                                      1
['05/2022']                                      2
['06/0140', '06/2022', '06/2022']                1
['06/2010', '03/2014']                           1
['06/2012', '05/2016']                           1
['06/2012']                                      1
['06/2014', '06/2018']                           1
['06

In [54]:
def date_formatting(dt_string):
    #dt_string = "Sep. 2021"
    # Convert string to datetime object
    try:
        dt_object = datetime.strptime(dt_string, "%m/%Y")
        return dt_object.strftime("%b %Y")
    except:
        try:
            dt_object = datetime.strptime(dt_string, "%b %Y")
            return dt_object.strftime("%b %Y")
        except:
            try:
                dt_object = datetime.strptime(dt_string, "%b. %Y")
                return dt_object.strftime("%b %Y")
            except:
                try:
                    dt_object = datetime.strptime(dt_string, "%b%Y")
                    return dt_object.strftime("%b %Y")
                except:
                    try:
                        dt_object = datetime.strptime(dt_string, "%b'%y")
                        return dt_object.strftime("%b %Y")
                    except:
                        try:
                            dt_object = datetime.strptime(dt_string, "%B %Y")
                            return dt_object.strftime("%b %Y")
                        except:
                            try:
                                dt_object = datetime.strptime(dt_string, "%B %Y")
                                return dt_object.strftime("%b %Y")
                            except:
                                pass

In [61]:
compared_df["master_graduation_date_x1"] = compared_df["master_graduation_date_x"].apply(lambda x:date_formatting(x))

In [115]:
compared_df["master_graduation_date_y1"] = compared_df["master_graduation_date_y"].apply(lambda x:date_formatting(str(x)[2:len(str(x))-2]))

In [128]:
comp_graduation_dates = compared_df.apply(lambda x: fuzz.ratio(x['master_graduation_date_x1'], x['master_graduation_date_y1']), axis = 1)
np.mean(comp_graduation_dates)

21.153846153846153

In [126]:
compared_df["bachelor_graduation_date_x1"] = compared_df["bachelor_graduation_date_x"].apply(lambda x:date_formatting(x))

In [125]:
compared_df["bachelor_graduation_date_y1"] = compared_df["bachelor_graduation_date_y"].apply(lambda x:date_formatting(str(x)[2:len(str(x))-2]))

In [127]:
comp_graduation_dates = compared_df.apply(lambda x: fuzz.ratio(x['bachelor_graduation_date_x1'], x['bachelor_graduation_date_y1']), axis = 1)
np.mean(comp_graduation_dates)

21.318681318681318